# **Project 2**
### Josh Winnes and Jon Contreras

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

import geopandas as gpd
import shapely
import skmob
from skmob.measures.individual import jump_lengths
from skmob.tessellation import tilers
from skmob.preprocessing import detection, clustering
from skmob.preprocessing import filtering

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 200)
import warnings
warnings.simplefilter("ignore")

import os

try:   
    os.chdir('/Users/joshwinnes/Library/Mobile Documents/com~apple~CloudDocs/Wheaton College/fall 2024/topics in data science/data')
    kmz_file_josh = '/Users/joshwinnes/Library/Mobile Documents/com~apple~CloudDocs/Wheaton College/fall 2024/topics in data science/data/March 7, 2024 - Whistler Blackcomb(Josh).kmz'
    #kmz_file_jon = '/Users/joshwinnes/Library/Mobile Documents/com~apple~CloudDocs/Wheaton College/fall 2024/topics in data science/data/March 7, 2024 - Whistler Blackcomb(Jon).kmz'
    
    import sys
    script_directory = '/Users/joshwinnes/Library/Mobile Documents/com~apple~CloudDocs/Wheaton College/fall 2024/topics in data science/Math485/Project2'
    sys.path.append(script_directory)
    from gpx_kmz_util import *
    
    print('Working from Josh\'s laptop')
except FileNotFoundError:
    os.chdir('/Users/jonathancontreras/Desktop/College Folder/Senior Year/Math485/Unit 1 Tabular/archive')
    kmz_file_josh = '/Users/jonathancontreras/Desktop/College Folder/Senior Year/Math485/Unit 1 Tabular/archive/data/March 7, 2024 - Whistler Blackcomb(Josh).kmz'
    kmz_file_jon = '/Users/jonathancontreras/Desktop/College Folder/Senior Year/Math485/Unit 1 Tabular/archive/data/March 7, 2024 - Whistler Blackcomb(Jon).kmz'
    
    import sys
    script_directory = 'path_to_your_directory_with_gpx_kmz_util'
    sys.path.append(script_directory)
    from gpx_kmz_util import *
    
    print('Working from Jonathan\'s laptop')

Working from Josh's laptop


In [2]:
df_jon_gpx = gpx_to_df('March 7, 2024 - Whistler Blackcomb(Jon).gpx')
df_josh_gpx = gpx_to_df('March 7, 2024 - Whistler Blackcomb(Josh).gpx')

kml_data_josh = extract_kml_from_kmz(kmz_file_josh)
df_josh_kmz = parse_kml_to_dataframe(kml_data_josh)

#kml_data_jon = extract_kml_from_kmz(kml_file_jon)
#df_jon_kml = parse_kml_to_dataframe(kml_data_jon)

In [3]:
df_jon_gpx.sort_values(by='Time')

,Latitude,Longitude,Elevation,Time
0,50.111385,-122.951516,699.522521,2024-03-07 09:52:51-08:00
1,50.109740,-122.953332,709.858664,2024-03-07 09:52:52-08:00
2,50.109765,-122.953274,741.954162,2024-03-07 09:52:53-08:00
3,50.109679,-122.953279,746.486242,2024-03-07 09:52:55-08:00
4,50.109586,-122.953275,748.005084,2024-03-07 09:52:58-08:00
...,...,...,...,...
3056,50.113127,-122.953518,671.599751,2024-03-07 16:19:06-08:00
3057,50.113150,-122.953668,671.413859,2024-03-07 16:19:10-08:00
3058,50.113189,-122.953797,668.187060,2024-03-07 16:19:28-08:00
3059,50.113201,-122.953949,670.080392,2024-03-07 16:19:35-08:00


In [4]:
df_josh_gpx.sort_values(by='Time')

,Latitude,Longitude,Elevation,Time
2474,50.058240,-122.919880,1566.485223,2024-03-07 01:00:02-08:00
2475,50.058248,-122.919729,1561.967929,2024-03-07 01:00:06-08:00
2476,50.058385,-122.919659,1562.047468,2024-03-07 01:00:10-08:00
2477,50.058496,-122.919682,1558.913951,2024-03-07 01:00:13-08:00
2478,50.058569,-122.919543,1557.256152,2024-03-07 01:00:21-08:00
...,...,...,...,...
2469,50.057834,-122.920106,1570.889197,2024-03-07 12:59:41-08:00
2470,50.057920,-122.919999,1570.976740,2024-03-07 12:59:53-08:00
2471,50.057920,-122.919999,1570.976740,2024-03-07 12:59:53-08:00
2472,50.058019,-122.920032,1570.891502,2024-03-07 12:59:55-08:00


In [5]:
print(df_josh_kmz.shape)
df_josh_kmz.drop('description', axis=1, inplace=True)
df_josh_kmz.columns = ['Name', 'Longitude', 'Latitude']
df_josh_kmz.head()

(25, 4)


,Name,Longitude,Latitude
0,Lift 1,-122.953366,50.113067
1,Run 1,-122.951056,50.078425
2,Lift 2,-122.963743,50.085441
3,Run 2,-122.947029,50.067096
4,Lift 3,-122.952236,50.066824


### ***Compiling Data into 1 DataFrame***

In [6]:
df_josh_kmz = pd.merge(df_josh_kmz[['Name', 'Longitude', 'Latitude']], df_josh_gpx, how = 'left', on=['Longitude', 'Latitude'])
df_josh_kmz.drop_duplicates(inplace=True)
print(df_josh_kmz.shape)
df_josh_kmz

(25, 5)


,Name,Longitude,Latitude,Elevation,Time
0,Lift 1,-122.953366,50.113067,691.382969,2024-03-07 09:49:35-08:00
1,Run 1,-122.951056,50.078425,1681.088996,2024-03-07 10:11:39-08:00
2,Lift 2,-122.963743,50.085441,1301.686260,2024-03-07 10:22:33-08:00
3,Run 2,-122.947029,50.067096,1855.330032,2024-03-07 10:36:24-08:00
5,Lift 3,-122.952236,50.066824,1779.142854,2024-03-07 10:43:17-08:00
6,Run 3,-122.957573,50.059251,2183.242267,2024-03-07 10:47:20-08:00
8,Lift 4,-122.964118,50.085837,1299.682279,2024-03-07 12:04:31-08:00
9,Run 4,-122.947552,50.066370,1853.043952,2024-03-07 12:16:09-08:00
10,Lift 5,-122.952248,50.066714,1772.088558,2024-03-07 12:19:03-08:00
11,Run 5,-122.957416,50.059398,2178.784326,2024-03-07 12:22:53-08:00


In [7]:
df_josh_kmz['time_elapsed'] = df_josh_kmz["Time"].diff(periods=-1)
df_josh_kmz['end_time'] = df_josh_kmz['Time']-df_josh_kmz['time_elapsed']
df_josh_kmz['time_elapsed'] = df_josh_kmz['end_time']-df_josh_kmz['Time']
df_josh_kmz

,Name,Longitude,Latitude,Elevation,Time,time_elapsed,end_time
0,Lift 1,-122.953366,50.113067,691.382969,2024-03-07 09:49:35-08:00,0 days 00:22:04,2024-03-07 10:11:39-08:00
1,Run 1,-122.951056,50.078425,1681.088996,2024-03-07 10:11:39-08:00,0 days 00:10:54,2024-03-07 10:22:33-08:00
2,Lift 2,-122.963743,50.085441,1301.686260,2024-03-07 10:22:33-08:00,0 days 00:13:51,2024-03-07 10:36:24-08:00
3,Run 2,-122.947029,50.067096,1855.330032,2024-03-07 10:36:24-08:00,0 days 00:06:53,2024-03-07 10:43:17-08:00
5,Lift 3,-122.952236,50.066824,1779.142854,2024-03-07 10:43:17-08:00,0 days 00:04:03,2024-03-07 10:47:20-08:00
6,Run 3,-122.957573,50.059251,2183.242267,2024-03-07 10:47:20-08:00,0 days 01:17:11,2024-03-07 12:04:31-08:00
8,Lift 4,-122.964118,50.085837,1299.682279,2024-03-07 12:04:31-08:00,0 days 00:11:38,2024-03-07 12:16:09-08:00
9,Run 4,-122.947552,50.066370,1853.043952,2024-03-07 12:16:09-08:00,0 days 00:02:54,2024-03-07 12:19:03-08:00
10,Lift 5,-122.952248,50.066714,1772.088558,2024-03-07 12:19:03-08:00,0 days 00:03:50,2024-03-07 12:22:53-08:00
11,Run 5,-122.957416,50.059398,2178.784326,2024-03-07 12:22:53-08:00,0 days 00:15:21,2024-03-07 12:38:14-08:00


In [8]:
df_josh_gpx = df_josh_gpx.merge(df_josh_kmz[['Name', 'Time']], how='left', on='Time')

In [9]:
df_josh_gpx.sort_values(by='Time')

,Latitude,Longitude,Elevation,Time,Name
2474,50.058240,-122.919880,1566.485223,2024-03-07 01:00:02-08:00,NaN
2475,50.058248,-122.919729,1561.967929,2024-03-07 01:00:06-08:00,NaN
2476,50.058385,-122.919659,1562.047468,2024-03-07 01:00:10-08:00,NaN
2477,50.058496,-122.919682,1558.913951,2024-03-07 01:00:13-08:00,NaN
2478,50.058569,-122.919543,1557.256152,2024-03-07 01:00:21-08:00,NaN
...,...,...,...,...,...
2469,50.057834,-122.920106,1570.889197,2024-03-07 12:59:41-08:00,NaN
2470,50.057920,-122.919999,1570.976740,2024-03-07 12:59:53-08:00,NaN
2471,50.057920,-122.919999,1570.976740,2024-03-07 12:59:53-08:00,NaN
2472,50.058019,-122.920032,1570.891502,2024-03-07 12:59:55-08:00,NaN


In [11]:
df_josh_gpx['Time'] = df_josh_gpx['Time'].apply(lambda x: x + pd.Timedelta(hours=12) if x.hour < 8 else x)
df_josh_gpx.sort_values(by='Time').head(20)

,Latitude,Longitude,Elevation,Time,Name
0,50.113260,-122.953950,683.549635,2024-03-07 09:46:36-08:00,NaN
1,50.113391,-122.953915,686.117031,2024-03-07 09:46:36-08:00,NaN
2,50.113319,-122.953772,688.737785,2024-03-07 09:48:27-08:00,NaN
3,50.113267,-122.953655,689.777724,2024-03-07 09:48:32-08:00,NaN
4,50.113181,-122.953594,692.476930,2024-03-07 09:49:04-08:00,NaN
5,50.113095,-122.953502,691.570411,2024-03-07 09:49:13-08:00,NaN
6,50.113067,-122.953366,691.382969,2024-03-07 09:49:35-08:00,Lift 1
7,50.112973,-122.953325,686.006277,2024-03-07 09:50:35-08:00,NaN
8,50.112877,-122.953391,681.919232,2024-03-07 09:51:15-08:00,NaN
9,50.112725,-122.953385,681.302449,2024-03-07 09:51:28-08:00,NaN


In [19]:
df_josh_gpx.fillna(method='ffill', axis=0, inplace=True)
df_josh_gpx.fillna(value = 'pre_lift_1', inplace = True)
df_josh_gpx.tail(30)


,Latitude,Longitude,Elevation,Time,Name
5496,50.110634,-122.953017,729.677547,2024-03-07 16:46:15-08:00,Run 12
5497,50.110750,-122.953040,726.598569,2024-03-07 16:46:16-08:00,Run 12
5498,50.110863,-122.953079,723.907668,2024-03-07 16:46:17-08:00,Run 12
5499,50.110986,-122.953094,722.526098,2024-03-07 16:46:18-08:00,Run 12
5500,50.111102,-122.953094,720.562804,2024-03-07 16:46:19-08:00,Run 12
5501,50.111227,-122.953101,716.086517,2024-03-07 16:46:20-08:00,Run 12
5502,50.111350,-122.953110,715.531140,2024-03-07 16:46:21-08:00,Run 12
5503,50.111471,-122.953065,714.977233,2024-03-07 16:46:22-08:00,Run 12
5504,50.111577,-122.953006,712.953679,2024-03-07 16:46:23-08:00,Run 12
5505,50.111683,-122.952964,709.429821,2024-03-07 16:46:24-08:00,Run 12
